### 3.2.3 Резонанс токов в параллельном контуре
Цель работы: исследование резонанса токов в параллельном колебательном контуре с изменяемой
ёмкостью, включающее получение амплитудно-частотных и фазово-частотных характеристик, а также определение основных параметров контура.\
В работе используются: генератор сигналов, источник тока, нагруженный на параллельный колебательный контур с переменной ёмкостью, двулучевой осциллограф, цифровые вольтметры.\
Схема установки, на которой будем проводить исследование резонанса в электрическом контуре, изображена на Рис. 1.
<img src="./3.2.3.png" />
На осциллограф подается напряжение генератора $E$ напряжение на контуре $U$ Параллельно контуру включен источник тока на операционном усилителе. Ток затвора полевого транзистора мал, и можно считать:
$$\tilde{I} = \frac{E}{R_1} =I_0 cos⁡(\omega t+\varphi_0 ), I_0=\frac{E_0}{R_1} $$
Активное сопротивление конденсатора:
$$R_s=\frac{tg(\delta)}{\omega C} ,   tg(\delta)<10^{-3}$$
Тогда суммарное сопротивление:
$$R_\sum=R_L+R+R_S$$
Резонансная частота:
$$\omega_0=\frac{1}{LC}$$
Волновое сопротивление:
$$\rho=\sqrt{\frac{L}{C}}$$
Добротность:
$$Q=\frac{\rho}{R_Σ} = \omega_0 \frac{L}{R_\sum} = \frac{1}{\omega_0 CR_\sum }$$

In [17]:
import numpy as m 
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display, HTML

Установим напряжение генератора $E = 96 мВ$. Измерим и запишем в таблицу резонансные частоты $f_n$ и соответствующие им напряжения $U_n$ для контуров с семью различными емкостями $C_n$. Также будем фиксировать изменение напряжения с генератора.
Последовательно вычислим все недостающие параметры: $L,\rho,Z_{рез},Q,R_\sum,R_{Smax},R_L$, и тоже запишем их в таблицу.
по формулам: $$L = \frac{1}{(2\pi f_n)^2C_n}$$
$$\rho = \sqrt{\frac{L}{C}} $$
$$Z_{рез} = \frac{U_n R_1}{E}$$
$$\sigma(\langle{X}\rangle)=\sqrt{\frac{1}{42}\Sigma(X_{i}-\langle{X}\rangle)^2}$$

In [72]:
data = pd.read_excel('3.2.3.xlsx').drop('Unnamed: 16',axis = 1)
scheme_char = data.iloc[:, 0:4].dropna()
R = 3.5 # в Омах
R_1 = 1008 # в Омах
def deviation_table(a):
    b = pd.DataFrame()
    b.loc['mean',a.name] = a.mean()
    b.loc['Random_deviation',a.name] = a.std()
    b.loc['Mean_square_deviation',a.name] = a.std()/a.size**0.5
    return b 
def surgery(k,*num,asc = False):
    d = k.copy()
    a = d[d.iloc[:,0]<=1]
    b = d[d.iloc[:,0]>1]
    if len(num) == 1:
        a.loc[a.shape[0]+1,a.keys()[1]] = num[0]
        b.loc[d.shape[0]+1,b.keys()[1]] = num[0]
    elif len(num) == 2:
        a.loc[a.shape[0]+1,a.keys()[1]] = num[0]
        b.loc[d.shape[0]+1,b.keys()[1]] = num[1]
    #print(a, b)
    a = a.set_index(a.keys()[1])
    b = b.set_index(b.keys()[1])
    a = a.sort_index()
    b = b.sort_index()
    a = a.interpolate(method='from_derivatives')
    b = b.interpolate(method='from_derivatives')
    b = b.sort_index(ascending=asc)
    #print(a, b)
    return [a, b]
def domega(d,num = round(0.5**0.5,3)):
    a, b = surgery(d,num)
    return b.loc[num]-a.loc[num]
def surgery_2(k):
    d = k.copy()
    a = d[d.iloc[:,0]<=1]
    b = d[d.iloc[:,0]>1]
    a.iloc[:,1] = (a.iloc[:,2]-a.iloc[:,1])/a.iloc[:,2] 
    b.iloc[:,1] = (b.iloc[:,2]+b.iloc[:,1])/b.iloc[:,2]
    #print(a, b)
    return pd.concat([b,a])

scheme_char['L_mcHn'] = (10**15)/(scheme_char.Cn_nF*(2*m.pi*scheme_char.fn)**2)
scheme_char['p'] = (scheme_char.L_mcHn*1000/scheme_char.Cn_nF)**0.5
scheme_char['Z рез'] = R_1*scheme_char.un_mV/scheme_char.en_mV
scheme_char['Q'] = scheme_char['Z рез']/scheme_char.p
scheme_char['R_sum'] = scheme_char.p/scheme_char.Q
scheme_char['R_s'] = scheme_char.p*10**-3
scheme_char['R_l'] = scheme_char.R_sum-scheme_char.R_s-R 
#print(scheme_char)
scheme_dev = deviation_table(scheme_char.L_mcHn)
scheme_dev = scheme_dev.join(deviation_table(scheme_char.R_l))
scheme_char
#scheme_dev.to_latex()
#sc = scheme_char.to_latex()

,Cn_nF,fn,un_mV,en_mV,L_mcHn,p,Z рез,Q,R_sum,R_s,R_l
0,25.1,32300.0,500.0,96.5,967.300689,196.310662,5222.797927,26.604759,7.378780,0.196311,3.682469
1,33.2,28100.0,390.0,96.5,966.250073,170.598704,4073.782383,23.879328,7.144200,0.170599,3.473602
2,47.3,23590.0,284.0,96.4,962.328263,142.636624,2969.626556,20.819524,6.851099,0.142637,3.208463
3,57.4,21430.0,238.0,96.3,960.912911,129.385637,2491.214953,19.254185,6.719871,0.129386,3.090485
4,67.5,19690.0,204.0,96.3,967.932469,119.748655,2135.327103,17.831742,6.715477,0.119749,3.095728
5,82.7,17841.0,168.8,96.3,962.268717,107.868696,1766.878505,16.379900,6.585430,0.107869,2.977562
6,101.6,16073.0,139.6,96.2,965.056335,97.460689,1462.752599,15.008642,6.493638,0.097461,2.896177


Для параметров катушки также вычислим среднее значение и ошибки:

In [73]:
scheme_dev

,L_mcHn,R_l
mean,964.578494,3.203498
Random_deviation,2.754303,0.280557
Mean_square_deviation,1.041029,0.106041


# Амплитудно-частотные характеристики
Для контуров с номерами 1 и 3 снимем в районе резонанса амплитудно-частотные характеристики и изобразим их на одном графике:

In [103]:
afc_data = data.iloc[:,4:8]
#fig = make_subplots(rows=1,cols=2)
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x = afc_data.fn_1/scheme_char.fn[0],
                          y=afc_data.un_1/scheme_char.un_mV[0],
                         mode='lines+markers',
                         name='U(Fr) при положении 1',
                         line_shape='spline'))
#              ,row = 1,col = 1)
fig2.add_trace(go.Scatter(x = afc_data.fn_3/scheme_char.fn[2],
                          y = afc_data.un_3/scheme_char.un_mV[2],
                         mode='lines+markers',
                         name='U(Fr) при положении 3',
                         line_shape='spline'))
#              ,row = 1,col = 2)
fig = go.Figure()
fig.add_trace(go.Scatter(x=afc_data.fn_1/1000, y=afc_data.un_1,
                mode='lines+markers',
                name='U(Fr) при положении 1',
                line_shape='spline'))
fig.add_trace(go.Scatter(x=afc_data.fn_3/1000, y=afc_data.un_3,
                         mode='lines+markers',
                         name='U(Fr) при положении 3',
                         line_shape='spline'))
display(afc_data)
fig.show()
#fig2 = make_subplots(rows=1,cols=2)


,fn_1,un_1,fn_3,un_3
0,31560,306,24339,174
1,31662,330,24207,194
2,31777,365,24121,210
3,31836,384,24016,220
4,31882,400,24009,230
5,31920,435,23932,245
6,32035,450,23840,260
7,32130,475,23745,275
8,32300,500,23590,284
9,32548,470,23425,270


Далее построим график отражающий функцию $\frac{U}{U_0} = f(\frac{f}{f_0})$

In [104]:
fig2.show()

Теперь вычислим добротность по формуле $$Q=\frac{\omega_0}{2\Delta \Omega}$$
при $\Delta \Omega$ взятом на уровне $\frac{U}{U_0}=\sqrt{\frac{1}{2}}$

In [100]:
u_1 = afc_data.un_1/scheme_char.un_mV[0]
uf_1 = afc_data[['fn_1']]/scheme_char.fn[0]
uf_1 = uf_1.join(u_1)
delta = domega(uf_1).to_numpy()
uf_1 = pd.concat(surgery(uf_1,round(1/0.5**0.5)))
u_2 = afc_data.un_3/scheme_char.un_mV[2]
uf_2 = afc_data[['fn_3']]/scheme_char.fn[2]
uf_2 = uf_2.join(u_2)
delta_2 = domega(uf_2).to_numpy()
uf_2 = pd.concat(surgery(uf_2,round(1/0.5**0.5)))
Q_graph = pd.DataFrame({'1-е положение':1/delta,'3-е положение':1/delta_2},index = ['fn'])

C:\Users\vanan\Miniconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\vanan\Miniconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [101]:
Q_graph

,1-е положение,3-е положение
fn,26.964175,29.213098


# Фазово-частотные характеристики
Для контуров с номерами 1 и 3 снимем в районе резонанса фазово-частотные характеристики и изобразим их на одном графике в координатах $x \equiv f/f_{0n}, y\equiv U/U_{0n} $ :

In [113]:
pfc_data = data.iloc[:,8:16].dropna()

display(pfc_data)

plot1_data = pfc_data.iloc[:,[0,2,3]]
plot1_data.loc[:,'fn_1.1'] = plot1_data['fn_1.1']/scheme_char.fn[0]
plot2_data = pfc_data.iloc[:,[4,6,7]]
plot2_data.loc[:,'fn_3.1'] = plot2_data['fn_3.1']/scheme_char.fn[2]
y_data_1 = surgery_2(plot1_data)
y_data_1 = y_data_1.drop('x0_1',axis=1)
y_data_1.loc[:,'ol'] = 1
y_data_1.loc[:,'type'] = '1-е положение'
y_data_2 = surgery_2(plot2_data)
y_data_2 = y_data_2.drop('x0_3',axis=1).rename(columns={"D_x_3": "D_x_1", "fn_3.1": "fn_1.1"})
y_data_2.loc[:,'ol'] = 1
y_data_2.loc[:,'type'] = '3-е положение'
squares = m.linalg.lstsq(y_data_1.iloc[:,[0,2]],y_data_1.iloc[:,1], rcond=-1)
squares2 = m.linalg.lstsq(y_data_2.iloc[:,[0,2]],y_data_2.iloc[:,1], rcond=-1)
y_data_1.loc[:,'deviation'] = y_data_1['fn_1.1']*float(squares[1])
y_data_2.loc[:,'deviation'] = y_data_2['fn_1.1']*float(squares2[1])
y_data_1.ol = 1/scheme_char.fn[0]
y_data_2.ol = 1/scheme_char.fn[2]
y_data = pd.concat([y_data_1,y_data_2])

fig1 = px.scatter(y_data, x = "fn_1.1",
                 y = "D_x_1", color= 'type',
                  
                 trendline = "ols",
                error_y = y_data['deviation'],
                error_x = y_data['ol'])
#                 height = 800, width = 800)
fig1.show()

,fn_1.1,un_1.1,D_x_1,x0_1,fn_3.1,un_3.1,D_x_3,x0_3
0,34320.0,150.0,3.0,7.5,25405.0,87.0,4.2,10.4
1,33750.0,195.0,2.9,7.5,24760.0,125.0,3.6,10.4
2,33474.0,233.0,2.7,7.5,24384.0,167.0,3.1,10.4
3,33164.0,292.0,2.4,7.5,24185.0,196.0,2.5,10.4
4,32990.0,336.0,2.1,7.5,24063.0,220.0,2.0,10.4
5,32817.0,387.0,1.8,7.5,23927.0,244.0,1.4,10.4
6,32647.0,436.0,1.4,7.5,23726.0,275.0,0.9,10.4
7,32300.0,500.0,0.0,7.5,23590.0,284.0,0.0,10.4
8,32020.0,441.0,1.1,7.5,23347.0,259.0,1.3,10.4
9,31860.0,390.0,1.5,7.5,23175.0,225.0,2.0,10.4


In [110]:
y_data_1 = pd.concat(surgery(y_data_1.iloc[:,[0,1]],0.75,1.25,asc = True))
y_data_2 = pd.concat(surgery(y_data_2.iloc[:,[0,1]],0.75,1.25,asc = True))
Q_1 = -1/(y_data_1.loc[0.75,'fn_1.1']-y_data_1.loc[1.25,'fn_1.1'])
Q_2 = -1/(y_data_2.loc[0.75,'fn_1.1']-y_data_2.loc[1.25,'fn_1.1'])

C:\Users\vanan\Miniconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\vanan\Miniconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\vanan\Miniconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [119]:
Q_graph.loc['PFC',:] = [Q_1,Q_2]
Q_graph.loc["Theoretically?",:] = [scheme_char.loc[0,'Q'],scheme_char.loc[2,'Q']]

Q_graph

,1-е положение,3-е положение
fn,26.964175,29.213098
PFC,32.291927,22.613836
Theoretically?,26.604759,20.819524
